<a href="https://colab.research.google.com/github/jfagudelov/ST0245-008/blob/master/ProyectoX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Referencias:
1. https://towardsdatascience.com/decision-trees-in-machine-learning-641b9c4e8052
2. https://www.geeksforgeeks.org/binary-tree-data-structure/
3. https://www.geeksforgeeks.org/decision-tree/
4. https://medium.com/swlh/making-data-trees-in-python-3a3ceb050cfd
5. https://www.educative.io/edpresso/binary-trees-in-python
6. https://www.freecodecamp.org/news/all-you-need-to-know-about-tree-data-structures-bceacb85490c/
7. https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as rd
from time import time

In [ ]:
'''
Here we upload the csv files with the data.
'''

#T0 = pd.read_csv("TEST 0A.csv", sep = ";")
T1 = pd.read_csv("TEST 1.csv", sep = ";")
#T2 = pd.read_csv("TEST 2.csv", sep = ";")
#T3 = pd.read_csv("TEST 3.csv", sep = ";")
#T4 = pd.read_csv("TEST 4.csv", sep = ";")
Tlite = pd.read_csv('TEST lite.csv', sep=";")
#Tr0 = pd.read_csv("TRAIN 0A.csv", sep = ";")
#Tr1 = pd.read_csv("TRAIN 1.csv", sep = ";")
#Tr2 = pd.read_csv("TRAIN 2.csv", sep = ";")
#Tr3 = pd.read_csv("TRAIN 3.csv", sep = ";")
#Tr4 = pd.read_csv("TRAIN 4.csv", sep = ";")

#test_data = pd.concat(T0,T1,T2,T3,T4)
#training_data = pd.concat(Tr0,Tr1,Tr2,Tr3,Tr4)

In [ ]:
# Filter
'''
This list will help us to filter the columns we intend to use.
'''
columns_to_use = ['estu_depto_reside.1', 'estu_mcpio_reside.1',
                  'fami_estratovivienda.1', 'fami_nivelsisben', 
                  'cole_depto_ubicacion',
                  'punt_lenguaje', 'punt_matematicas',
                  'punt_biologia', 'punt_quimica', 'punt_fisica',
                  'punt_ciencias_sociales', 'punt_filosofia', 'punt_ingles',
                  'desemp_ingles', 'exito']

# Data Exploration

In [ ]:
Tlite.groupby(pd.Series(Tlite.exito)).agg({'punt_matematicas': 'mean'}).plot()

In [ ]:
Tlite.groupby(pd.Series(Tlite.exito)).agg({'punt_filosofia': 'mean'}).plot()

In [ ]:
Tlite.groupby(pd.Series(Tlite.exito)).agg({'punt_quimica': 'mean'}).plot(kind='bar')

In [ ]:
Tlite.groupby(pd.Series(Tlite.exito)).agg({'punt_matematicas': 'mean'}).plot(kind='bar')

In [ ]:
Tlite.rename(columns={'estu_depto_reside.1':'estu_depto_reside'})

In [ ]:
Tlite.rename(columns={'estu_genero.1':'estu_genero'})

In [ ]:
Tlite0 = Tlite[Tlite['exito'] == 0] 

In [ ]:
Tlite1 = Tlite[Tlite['exito'] == 1]     

# Construction methods

In [ ]:
'''
This method returns a new dataframe without nan values
and with the columns defined in columns_to_use.
'''

def filter_df(df, columns_to_use):
  df.dropna(how='all',axis=0, inplace=True)
  new_df = df.loc[:, columns_to_use]
  return new_df

In [ ]:
filtered_df = filter_df(Tlite, columns_to_use ) 

In [ ]:
def is_value(value):
  ''' 
  This method verifies if a value is an int or float.
  '''
  return isinstance(value, int) or isinstance(value, float)

In [ ]:
def class_counts(df):
  '''
  This methods returns a dictionary with
  the number of successess and fails in a dataframe.
  '''
  labels = {}
  for value in df.loc[:, 'exito']:
    if value not in labels:
      labels[value] = 0
    labels[value] += 1
  return labels

class_counts(a[1])

In [ ]:
class Question():
  '''
  This class creates the questions for the tree nodes.
  '''
  def __init__(self, column, value):
    self.column = column
    self.value = value
  

  '''
  This method verifies if a value is true or not
  to the condition of a question.
  '''
  def match(self, val):
    if is_value(val):
      return val >= self.value
    else:
      return val == self.value
  
  def __repr__(self):
    condition = '=='
    if is_value(self.value):
      condition = '>='
    return 'Is {} {} {}?'.format(self.column, condition, self.value)

In [ ]:
'''
This method return a true_rows dataframe and a false_rows dataframe,
whereas true_rows contains the rows from the original dataframe that
are true to the question and false_rows the ones that not.
'''

def partition(df, question):
  true_rows, false_rows = pd.DataFrame(columns=df.columns), pd.DataFrame(columns=df.columns)
  for row in range(len(df)):
    actual_row = df.loc[row, df.columns].to_dict()
    if question.match(df.loc[row, question.column]):
      true_rows = true_rows.append(actual_row, ignore_index=True)
    else:
      false_rows = false_rows.append(actual_row, ignore_index=True)
  return true_rows, false_rows

partition(filtered_df, Question('punt_ingles', 36.0))

In [ ]:
'''
This method calculates the Gini impurity of a node.
'''

def gini(df):
  # This method calculates the gini impurity
  labels = class_counts(df)
  impurity = 1
  for value in labels:
    probability = labels[value]/len(df)
    impurity -= probability**2
  return impurity

In [ ]:
'''
This method calculates the information gain from the true and false rows.
'''

def info_gain(left, right): 
  return ((len(left)*gini(left)+(len(right)*gini(right)))/(len(left)+len(right)))

In [ ]:
'''
This method finds the best question for a node given its current dataframe.
'''

def find_best_partition(df):
  best_gain = 0
  best_question = None

  for column in df.columns:
    if column == 'exito':
      continue
    values = [row for row in df.loc[:, column]]
    for value in values:

      question = Question(column, value)
      success_rows, fail_rows = partition(df, question)
      if len(success_rows) == 0 or len(fail_rows) == 0:
        continue

      gain = info_gain(success_rows, fail_rows)

      if gain >= best_gain:
        best_gain, best_question = gain, question

  return best_gain, best_question

# Tree building

In [ ]:
'''
This class will create a leaf for the tree that will contain tree's predictions
'''

class Leaf():

  def __init__(self, df):
    self.predictions = class_counts(df)

In [ ]:
'''
This class will create the nodes of the tree, and will contain the node's
true and false branch and it's question.
'''

class Decision_Node():

  def __init__(self, question, true_branch, false_branch):
      self.question = question
      self.true_branch = true_branch
      self.false_branch = false_branch

This tree will be built recursively, finding for each new node (including the initial node) the best question to create the next child nodes. This will be done until the information gain of a node is 0, then will be a leaf and recursion ends.

In [3]:
def build_tree(df):
  gain, question = find_best_partition(df)
  print(gain, question)
  if gain == 0:
    return Leaf(df)
  true_r, false_r = partition(df, question)
  print(true_r, false_r)
  true_b = build_tree(true_r)
  false_b = build_tree(false_r)
  return Decision_Node(question, true_b, false_b)

In [6]:
class Tree():

  def __init__(self, df, inf=0, sup=None):
    if inf != 0:
      if sup != None:
        df = df.loc[inf:sup, df.columns]
      else:
        df = df.loc[inf:, df.columns]
    self.tree = build_tree(df)

  '''
  This method will be used to predict the success a of a value
  '''

  def classify(df, node):
    if isinstance(node, Leaf):
      return node.predictions

    if node.question.match(df):
      return classify(df, node.true_branch)
    else:
      return classify(df, node.false_branch)

  def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

      # Base case: we've reached a leaf
    if isinstance(node, Leaf):
      return type(node)

      # Print the question at this node
    print(spacing + str(node.question))

      # Call this function recursively on the true branch
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

      # Call this function recursively on the false branch
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [ ]:
def random_trees(df, n):
  trees = []
  success
  for n in range(n):
    lim_inf, lim_sup = rd.randint(0, len(df)//2), rd.randint(len(df)//2, len(df))
    trees.append(Tree(df, lim_inf, lim_sup))
  

# Testing

In [ ]:
build_tree(filtered_df)

In [ ]:
print_tree(tree_1)